# CAPSTONE PROJECT - The Battle Of Neighborhoods

## CODE

### Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    openssl:       1.0.2p-h470a237_1 conda-forge --> 1.0.2p-h470a237_2 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
ope

### First we work on Toronto

#### We have explored and sorted data regarding neighborhoods of Downtown Toronto in one of our previous
#### assignments. So we will select "Downtown Toronto" as a Borough and anylze its neighborhoods.

In [2]:
path='neigbors_geographical.csv'
df=pd.read_csv(path, index_col=0)

In [3]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Sorting according to Downtown Toronto

In [4]:
# set index for only Downtown Toronto
downtown_toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
downtown_toronto_data=downtown_toronto_data.drop(['Postcode'], axis=1)
downtown_toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383


### Define Foursquare Credentials and Version

In [5]:
CLIENT_ID = 'HRMBKZUASN1NWO005IQK4TGG15UVEY5GCLJCYXHXW0VDP00K' # your Foursquare ID
CLIENT_SECRET = 'JSXFO23NR2OMICQSZRFQYDAZG1GMNRALXXACAFVNF5CGAM4C' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:HRMBKZUASN1NWO005IQK4TGG15UVEY5GCLJCYXHXW0VDP00K
CLIENT_SECRET:JSXFO23NR2OMICQSZRFQYDAZG1GMNRALXXACAFVNF5CGAM4C


#### Get the geographical coordinates of Downtown Toronto

In [6]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


43.655115 -79.380219


### Visualization

In [7]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

### Explore Neighborhoods in Downtown Toronto

#### Let's create a function to repeat the process to all the neighborhoods in Toronto

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [9]:
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


#### Let's check the size of the resulting dataframe

In [10]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(334, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Let's check how many venues were returned for each neighborhood

In [11]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
"Chinatown,Grange Park,Kensington Market",20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
"Commerce Court,Victoria Hotel",20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues

In [12]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 121 uniques categories.


### Analyzing Each Neighborhood

In [13]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Vietnamese Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Ethiopian Restaurant,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Gastropub,General Entertainment,General Travel,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Video Game Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### New dataframe size

In [14]:
downtown_toronto_onehot.shape

(334, 121)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [15]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped

,Neighborhood,Vietnamese Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Ethiopian Restaurant,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Gastropub,General Entertainment,General Travel,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Video Game Store
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.00,0.00,0.00,0.1,0.0000,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0500,0.00,0.00,0.00,0.00,0.00,0.00,0.0500,0.00,0.00,0.00,0.05,0.0000,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.0000,0.00,0.05,0.000000,0.00,0.00,0.05,0.00,0.00,0.0,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.05,0.05,0.00,0.000,0.00,0.00,0.00,0.05,0.000000,0.00,0.05,0.00,0.00,0.0000,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.05,0.00,0.00,0.10,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.0,0.0000,0.00,0.0000,0.00,0.00,0.05,0.05,0.05,0.05,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.10,0.0000,0.00,0.00,0.00,0.05,0.0000,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.10,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.0500,0.00,0.05,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.0000,0.00,0.00,0.00,0.050,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.05,0.00,0.0000,0.00,0.00,0.00,0.000000,0.10,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.00,0.00,0.00,0.00,0.0,0.0000,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.071429,0.00,0.071429,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.071429,0.00,0.00,0.00,0.00,0.00,0.0,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.071429,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0.071429,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,"Cabbagetown,St. James Town",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.0,0.0000,0.00,0.0000,0.05,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.05,0.1000,0.05,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.00,0

#### New size

In [16]:
downtown_toronto_grouped.shape

(18, 121)

#### Let's print each neighborhood along with the top 5 most common venues

In [17]:
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0    Asian Restaurant  0.10
1          Steakhouse  0.10
2        Concert Hall  0.05
3  Seafood Restaurant  0.05
4         Coffee Shop  0.05


----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1        Cocktail Bar  0.10
2      Farmers Market  0.10
3            Beer Bar  0.05
4           Jazz Club  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3   Harbor / Marina  0.07
4           Airport  0.07


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.15
1                Café  0.10
2   Indian Restaurant  0.10
3               Diner  0.05
4  Italian Restaurant  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.05
2      Sandwich Place  0.0

#### Let's put that into a pandas dataframe

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Asian Restaurant,Steakhouse,Food Court,Speakeasy,Noodle House,Opera House,Pizza Place,Coffee Shop,Plaza,Concert Hall
1,Berczy Park,Seafood Restaurant,Farmers Market,Cocktail Bar,Park,Concert Hall,Italian Restaurant,Jazz Club,Liquor Store,Museum,Bistro
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Boutique,Harbor / Marina
3,"Cabbagetown,St. James Town",Restaurant,Indian Restaurant,Café,General Entertainment,Bakery,Butcher,Pet Store,Pub,Jewelry Store,Gastropub
4,Central Bay Street,Coffee Shop,Park,Spa,Bubble Tea Shop,Modern European Restaurant,Chinese Restaurant,Pizza Place,Italian Restaurant,Ramen Restaurant,Bar
5,"Chinatown,Grange Park,Kensington Market",Café,Caribbean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Organic Grocery,Arts & Crafts Store,Italian Restaurant,Dessert Shop,Bar,Bakery
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Coffee Shop,Restaurant,Convenience Store,Baby Store,Italian Restaurant
7,Church and Wellesley,Park,Mexican Restaurant,Ramen Restaurant,Restaurant,Japanese Restaurant,Salon / Barbershop,Bar,Bookstore,Pizza Place,Piano Bar
8,"Commerce Court,Victoria Hotel",Café,Gastropub,Deli / Bodega,Museum,Coffee Shop,Pub,Restaurant,Japanese Restaurant,Bakery,Hotel
9,"Design Exchange,Toronto Dominion Centre",Café,Coffee Shop,Deli / Bodega,Gastropub,Hotel,Pub,Restaurant,Japanese Restaurant,Bakery,Hotel Bar


### Cluster Neighborhoods

In [20]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 3, 1, 3, 3, 0, 3, 3], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [21]:
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Breakfast Spot,Historic Site,Performing Arts Venue,Chocolate Shop,Mexican Restaurant,Pub,Restaurant,Spa
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Café,Sandwich Place,American Restaurant,Burger Joint,Clothing Store,Pizza Place,Beer Bar,Plaza,Ramen Restaurant,Comic Shop
2,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Hotel,Performing Arts Venue,Church,Middle Eastern Restaurant,Cosmetics Shop,Creperie
3,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Seafood Restaurant,Farmers Market,Cocktail Bar,Park,Concert Hall,Italian Restaurant,Jazz Club,Liquor Store,Museum,Bistro
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Park,Spa,Bubble Tea Shop,Modern European Restaurant,Chinese Restaurant,Pizza Place,Italian Restaurant,Ramen Restaurant,Bar


#### Finally, let's visualize the resulting clusters

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
#### Based on the defining categories, we can then assign a name to each cluster.

### Commercial Activity (Including Tourist Places)  

In [23]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",Coffee Shop,Bakery,Breakfast Spot,Historic Site,Performing Arts Venue,Chocolate Shop,Mexican Restaurant,Pub,Restaurant,Spa
1,"Ryerson,Garden District",Café,Sandwich Place,American Restaurant,Burger Joint,Clothing Store,Pizza Place,Beer Bar,Plaza,Ramen Restaurant,Comic Shop
7,"Harbourfront East,Toronto Islands,Union Station",Hotel,Park,Café,Italian Restaurant,Salad Place,Bakery,Plaza,Skating Rink,Sporting Goods Shop,Sports Bar
11,"Chinatown,Grange Park,Kensington Market",Café,Caribbean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Organic Grocery,Arts & Crafts Store,Italian Restaurant,Dessert Shop,Bar,Bakery
12,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Boutique,Harbor / Marina
15,"Cabbagetown,St. James Town",Restaurant,Indian Restaurant,Café,General Entertainment,Bakery,Butcher,Pet Store,Pub,Jewelry Store,Gastropub
17,Church and Wellesley,Park,Mexican Restaurant,Ramen Restaurant,Restaurant,Japanese Restaurant,Salon / Barbershop,Bar,Bookstore,Pizza Place,Piano Bar


### Residential With Services

In [24]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Coffee Shop,Park,Spa,Bubble Tea Shop,Modern European Restaurant,Chinese Restaurant,Pizza Place,Italian Restaurant,Ramen Restaurant,Bar
13,Rosedale,Park,Trail,Playground,Video Game Store,Concert Hall,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Church


### Commercial Hub

In [25]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Stn A PO Boxes 25 The Esplanade,Farmers Market,Cocktail Bar,Café,Park,Restaurant,Food Truck,Hotel,Concert Hall,Jazz Club,Museum


### Cultural & Getting Out Places

In [26]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Berczy Park,Seafood Restaurant,Farmers Market,Cocktail Bar,Park,Concert Hall,Italian Restaurant,Jazz Club,Liquor Store,Museum,Bistro
5,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Coffee Shop,Restaurant,Convenience Store,Baby Store,Italian Restaurant
6,"Adelaide,King,Richmond",Asian Restaurant,Steakhouse,Food Court,Speakeasy,Noodle House,Opera House,Pizza Place,Coffee Shop,Plaza,Concert Hall
8,"Design Exchange,Toronto Dominion Centre",Café,Coffee Shop,Deli / Bodega,Gastropub,Hotel,Pub,Restaurant,Japanese Restaurant,Bakery,Hotel Bar
9,"Commerce Court,Victoria Hotel",Café,Gastropub,Deli / Bodega,Museum,Coffee Shop,Pub,Restaurant,Japanese Restaurant,Bakery,Hotel
10,"Harbord,University of Toronto",Bakery,Japanese Restaurant,Bookstore,Restaurant,Video Game Store,Italian Restaurant,Comfort Food Restaurant,College Gym,Chinese Restaurant,Café
16,"First Canadian Place,Underground city",Café,Coffee Shop,Steakhouse,Gastropub,Gym / Fitness Center,Pizza Place,Pub,Restaurant,Seafood Restaurant,Deli / Bodega


### Activity Center

In [27]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Hotel,Performing Arts Venue,Church,Middle Eastern Restaurant,Cosmetics Shop,Creperie


### Saving the file for future purpose

In [28]:
downtown_toronto_merged.to_csv('downtown_toronto_mostcommonvenues.csv')

### Now we will move towards New York

#### We have already explored and sorted data regarding neighborhoods of five Boroughs in New York in one of our previous
#### assiignments. So we will select "Manhattan" as a Borough and anylze its neighborhoods.

In [29]:
path1='neighborhoods_NY.csv'
neighborhoods=pd.read_csv(path1, index_col=0)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [30]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use geopy library to get the latitude and longitude values of New York City.

In [31]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


### Create a map of New York with neighborhoods superimposed on top.

In [32]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to 
#### reveal the name of the neighborhood and its respective borough.

#### However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. 
#### So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [33]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Let's get the geographical coordinates of Manhattan.

In [34]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


#### As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [35]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues

In [37]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check the size of the resulting dataframe

In [38]:
print(manhattan_venues.shape)
manhattan_venues.head()

(798, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


#### Let's check how many venues were returned for each neighborhood

In [39]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues

In [40]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 203 uniques categories.


### Analyzing the Neighborhoods

In [41]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### And let's examine the new dataframe size.

In [42]:
manhattan_onehot.shape

(798, 204)

In [43]:
#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Battery Park City,0.00,0.00,0.0,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.050000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.05,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.00,0.00,0.05,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.05,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.150000,0.05,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,0.00,0.00,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0

#### Let's confirm the new size

In [45]:
manhattan_grouped.shape

(40, 204)

#### Let's print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0            Park  0.15
1      Smoke Shop  0.05
2      Food Court  0.05
3  Scenic Lookout  0.05
4     Coffee Shop  0.05


----Carnegie Hill----
                  venue  freq
0                   Gym  0.10
1  Gym / Fitness Center  0.10
2    Italian Restaurant  0.10
3                   Spa  0.10
4      Ramen Restaurant  0.05


----Central Harlem----
                 venue  freq
0   African Restaurant  0.10
1  American Restaurant  0.10
2         Dessert Shop  0.05
3                  Bar  0.05
4                 Café  0.05


----Chelsea----
                venue  freq
0             Theater  0.10
1  Italian Restaurant  0.10
2           Nightclub  0.10
3    Asian Restaurant  0.10
4                Café  0.05


----Chinatown----
                venue  freq
0        Noodle House  0.10
1  Chinese Restaurant  0.10
2         Yoga Studio  0.05
3                 Bar  0.05
4  English Restaurant  0.05


----Civic Center----
                   venue  freq

#### Let's put that into a pandas dataframe

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Food Court,Market,Performing Arts Venue,Department Store,Plaza,Coffee Shop,Sandwich Place,Scenic Lookout,Shopping Mall
1,Carnegie Hill,Italian Restaurant,Spa,Gym,Gym / Fitness Center,Community Center,Café,Bookstore,Karaoke Bar,Gourmet Shop,Coffee Shop
2,Central Harlem,African Restaurant,American Restaurant,Juice Bar,Jazz Club,Caribbean Restaurant,Chinese Restaurant,Music Venue,Ethiopian Restaurant,Library,Beer Bar
3,Chelsea,Asian Restaurant,Nightclub,Theater,Italian Restaurant,Indian Restaurant,New American Restaurant,Café,French Restaurant,Beer Bar,Gift Shop
4,Chinatown,Noodle House,Chinese Restaurant,Yoga Studio,Tea Room,Greek Restaurant,General Entertainment,Garden Center,English Restaurant,Museum,Pizza Place
5,Civic Center,Bakery,Falafel Restaurant,Yoga Studio,Cuban Restaurant,Park,Coffee Shop,Nail Salon,Monument / Landmark,Molecular Gastronomy Restaurant,Spa
6,Clinton,Theater,Gym / Fitness Center,Pie Shop,Building,Café,French Restaurant,Mediterranean Restaurant,Lounge,Dog Run,Comedy Club
7,East Harlem,Mexican Restaurant,Latin American Restaurant,Pet Store,Pharmacy,Beer Bar,Donut Shop,Sandwich Place,Steakhouse,Thai Restaurant,Bakery
8,East Village,Coffee Shop,Ice Cream Shop,Hot Dog Joint,Japanese Restaurant,Park,Dog Run,Moroccan Restaurant,Cheese Shop,Burger Joint,Speakeasy
9,Financial District,Coffee Shop,Steakhouse,Jewelry Store,Gym / Fitness Center,Hotel,Pizza Place,Salad Place,New American Restaurant,Event Space,Monument / Landmark


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [50]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 2, 2, 2, 3, 2, 2, 1], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Coffee Shop,Yoga Studio,Gym,Shoe Store,Seafood Restaurant,Steakhouse,Supplement Shop,Donut Shop,Diner
1,Manhattan,Chinatown,40.715618,-73.994279,3,Noodle House,Chinese Restaurant,Yoga Studio,Tea Room,Greek Restaurant,General Entertainment,Garden Center,English Restaurant,Museum,Pizza Place
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Park,Italian Restaurant,Restaurant,Tapas Restaurant,Frozen Yogurt Shop,Ramen Restaurant,Cocktail Bar,Bakery,Caribbean Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,2,Wine Bar,Park,Café,Yoga Studio,Coffee Shop,Farmers Market,Spanish Restaurant,Bistro,Frozen Yogurt Shop,Latin American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Yoga Studio,Mexican Restaurant,Caribbean Restaurant,Cocktail Bar,Japanese Restaurant,Monument / Landmark,Mediterranean Restaurant,Café,Smoke Shop,Coffee Shop


#### manhattan_merged.shape

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
#### Based on the defining categories, we can then assign a name to each cluster.

#### Residential 

In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Village,Cosmetics Shop,Chinese Restaurant,Italian Restaurant,Gastropub,Gourmet Shop,Accessories Store,Bakery,Coffee Shop,Cocktail Bar,Candy Store


#### Commercial Place

In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Gym,Shoe Store,Seafood Restaurant,Steakhouse,Supplement Shop,Donut Shop,Diner
9,Yorkville,Wine Shop,Deli / Bodega,Hobby Shop,Italian Restaurant,Monument / Landmark,Liquor Store,Sandwich Place,Beer Store,Sushi Restaurant,Dog Run
10,Lenox Hill,Gym,Wine Shop,Burger Joint,Gym / Fitness Center,Salad Place,Taco Place,Dessert Shop,Gourmet Shop,Thai Restaurant,College Academic Building
11,Roosevelt Island,Sandwich Place,Rental Car Location,Coffee Shop,Farmers Market,School,Liquor Store,Dog Run,Residential Building (Apartment / Condo),Baseball Field,Bus Line
18,Greenwich Village,Sushi Restaurant,Italian Restaurant,Yoga Studio,Jazz Club,Gourmet Shop,French Restaurant,Mexican Restaurant,New American Restaurant,Optical Shop,Coffee Shop
22,Little Italy,Café,Wine Bar,Ice Cream Shop,Sandwich Place,Spanish Restaurant,Coffee Shop,Seafood Restaurant,Salon / Barbershop,Salad Place,Snack Place
34,Sutton Place,Indian Restaurant,Grocery Store,Gym,Steakhouse,Adult Boutique,American Restaurant,Beer Garden,Beer Store,Dessert Shop,Greek Restaurant


#### Tourist Areas & Hubs

In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Wine Bar,Park,Café,Yoga Studio,Coffee Shop,Farmers Market,Spanish Restaurant,Bistro,Frozen Yogurt Shop,Latin American Restaurant
4,Hamilton Heights,Yoga Studio,Mexican Restaurant,Caribbean Restaurant,Cocktail Bar,Japanese Restaurant,Monument / Landmark,Mediterranean Restaurant,Café,Smoke Shop,Coffee Shop
5,Manhattanville,Italian Restaurant,Indian Restaurant,Japanese Curry Restaurant,Other Nightlife,Ramen Restaurant,Dumpling Restaurant,Climbing Gym,Mexican Restaurant,Seafood Restaurant,Lounge
7,East Harlem,Mexican Restaurant,Latin American Restaurant,Pet Store,Pharmacy,Beer Bar,Donut Shop,Sandwich Place,Steakhouse,Thai Restaurant,Bakery
8,Upper East Side,Hotel,Bakery,Sandwich Place,Burrito Place,Spa,Chocolate Shop,Bridal Shop,Boutique,Bookstore,Grocery Store
12,Upper West Side,American Restaurant,Italian Restaurant,Bar,Chinese Restaurant,Juice Bar,Gift Shop,Pub,Bookstore,Seafood Restaurant,Bakery
13,Lincoln Square,Concert Hall,Indie Movie Theater,Theater,Opera House,Performing Arts Venue,Park,School,Library,Gift Shop,Fountain
15,Midtown,Hotel,Park,Smoke Shop,Grocery Store,Steakhouse,Chinese Restaurant,French Restaurant,Christmas Market,Sporting Goods Shop,Mediterranean Restaurant
16,Murray Hill,Burger Joint,Italian Restaurant,Hotel,Coffee Shop,Spa,Speakeasy,Museum,Restaurant,Taco Place,Tea Room
19,East Village,Coffee Shop,Ice Cream Shop,Hot Dog Joint,Japanese Restaurant,Park,Dog Run,Moroccan Restaurant,Cheese Shop,Burger Joint,Speakeasy


#### Center Acivity

In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Noodle House,Chinese Restaurant,Yoga Studio,Tea Room,Greek Restaurant,General Entertainment,Garden Center,English Restaurant,Museum,Pizza Place
2,Washington Heights,Café,Park,Italian Restaurant,Restaurant,Tapas Restaurant,Frozen Yogurt Shop,Ramen Restaurant,Cocktail Bar,Bakery,Caribbean Restaurant
6,Central Harlem,African Restaurant,American Restaurant,Juice Bar,Jazz Club,Caribbean Restaurant,Chinese Restaurant,Music Venue,Ethiopian Restaurant,Library,Beer Bar
14,Clinton,Theater,Gym / Fitness Center,Pie Shop,Building,Café,French Restaurant,Mediterranean Restaurant,Lounge,Dog Run,Comedy Club
25,Manhattan Valley,Bar,Yoga Studio,Arts & Crafts Store,Chinese Restaurant,Korean Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Fried Chicken Joint,Italian Restaurant
35,Turtle Bay,Sushi Restaurant,Karaoke Bar,Cocktail Bar,Duty-free Shop,Museum,Tennis Court,Farmers Market,Noodle House,Greek Restaurant,Grocery Store
36,Tudor City,Park,Yoga Studio,Thai Restaurant,Convenience Store,Deli / Bodega,Pizza Place,Coffee Shop,Dog Run,Mexican Restaurant,Spa


#### Cultural & Going Out Places

In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Asian Restaurant,Nightclub,Theater,Italian Restaurant,Indian Restaurant,New American Restaurant,Café,French Restaurant,Beer Bar,Gift Shop
